In [1]:
import os
from enum import Enum
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
from z_base import Experiment, Mols, T_ZERO
from scipy.stats import linregress

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.99
plt.rcParams['figure.subplot.bottom'] = 0.1


In [2]:
%matplotlib qt

In [7]:
DF=pd.read_excel(r'Datas\All_viscosity.xlsx',index_col=0)

In [4]:
class W(Enum):
    w = "w mol  %"
    w_mass = "w mass  %"
    w_vol = "w vol  %"
    T = 'Temperature  C'
    Nu = 'Viscosity  cP'
    ln_Nu = 'log Viscosity  cP'
    V = 'Kinematic viscosity cCt'
    ln_D = 'log Diffusion'
    D = 'Diffusion'
    T_1 = '1000/Temperature  K$^{-1}$'

In [8]:
# Comparation pure
x_i = W.T
y_i = W.V

compound_filter = (DF["w"] == 100) | (DF["w"] == 0)
df =DF[compound_filter]
data = []

fig, ax = plt.subplots(num="DT")
for (compound, *_), group in df.groupby(by=['compound']):
    y = group[y_i.name]
    T = group[x_i.name]
    ax.scatter(T, y, s=5,label=f"{compound:5}")

    # R = linregress(T, y)
    # line_x = np.array([0.00315, 0.0035])
    # ax.plot(
    #     line_x*1000,
    #     R.slope * line_x + R.intercept,
    #     linewidth=1,
    #     label=f"{compound:5}",
    # )
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [10]:
# Grouped compound Y(x)
compound = Mols.ocm
x_i = W.T_1
y_i = W.ln_Nu
g_i = W.w_mass

compound_filter = (DF["compound"] == compound.name)
# | (DF["compound"] == Mols.butanol.name)


fig, ax = plt.subplots(num="DT")
grouped = DF[compound_filter].groupby(by=[g_i.name])
colors_list = mcp.gen_color(cmap="brg", n=len(grouped))
for _, group in grouped:
    y = group[y_i.name]
    T = group[x_i.name]
    color = colors_list.pop()

    ax.scatter(T*1000, y, c=color, s=2)

    R = linregress(T, y)
    line_x = np.array([0.00315, 0.0035])
    ax.plot(
        line_x*1000,
        R.slope * line_x + R.intercept,
        c=color,
        linewidth=1,
        label=f"{group[g_i.name].mean(): 5.2f} %",
    )
# ax.set_title(compound.name.upper())
# ax.set_xlim(15, 40)
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [29]:
# Window Y(x)
compound = Mols.ocm
x_i = W.w
y_i = W.ln_Nu

compound_filter = (DF["compound"] == compound.name)|(DF["compound"] == Mols.butanol.name)
# | (DF["compound"] == Mols.butanol.name)
grouped = DF[compound_filter]
temper_list = np.arange(15, 41, 5)+T_ZERO
colors_list = mcp.gen_color(cmap="jet", n=len(temper_list))

fig, ax = plt.subplots(num="DX")
for T in temper_list:
    cond = (T - 2.5 <= grouped["T"]) & (grouped["T"] <= T + 2.5)
    group2= grouped[cond]
    group = group2[[x_i.name, y_i.name]].groupby(by=x_i.name).mean().reset_index()

    color = colors_list.pop()
    x = group[x_i.name]
    y = group[y_i.name]
    ax.scatter(x, y, c=color, label=f"{T-T_ZERO: 3.0f} C")
    line_x=np.array([0,60])
    r=linregress(x[:-1],y[:-1])
    ax.plot(line_x,r.slope*line_x+r.intercept,c=color,)

# ax.set_title(f"{compound.value.upper()}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel(y_i.value)
plt.legend()
plt.show()

In [56]:
# E
x_i = W.w
y_i = W.ln_Nu
shift = 17.24

fig, ax = plt.subplots(num="E")
for (compound, *_), group in DF.groupby(by=["compound"]):
    ys = []
    xs = []
    for (x_val, *_), sub_group in group.groupby(by=[x_i.name]):
        x = np.array(sub_group['T_1'])
        y = np.array(sub_group[y_i.name])
        y = np.log(10**y)
        r = linregress(x, y)

        ys.append(r.slope * 8.314 / 1000)
        xs.append(x_val)
    ax.plot([xs[0], 100], [ys[0], shift])
    ax.scatter(xs, ys, label=f"{compound}")

# ax.set_title(f"E {y_i.value}")
ax.set_xlabel(f"{x_i.value}")
ax.set_ylabel("E  kJ/mol")
plt.legend()
plt.show()

In [42]:
# Table
g_i = W.w_mass

x_i = W.T_1
y_i = W.ln_Nu

y2_i = W.ln_D
data = []
for (compound, *_), group in DF.groupby(by=["compound"]):
    for (x_val, *_), sub_group in group.groupby(by=[g_i.name]):
        T = sub_group[x_i.name]
        y = sub_group[y_i.name].array
        r = linregress(T, y)
        E_vis = r.slope * 8.314 / 1000
        v0 = 10**r.intercept

        rd = linregress(sub_group[[x_i.name, y2_i.name]])
        E_dif = -rd.slope * 8.314 / 1000
        rho = sub_group['rho'].mean()
        data.append(
            {
                'compound': compound,
                'x mol': round(x_val, 1),
                'rho [g/ml]': round(rho, 2),
                'E vis [kJ/mol]': round(E_vis, 2),
                'E dif [kJ/mol]': round(E_dif, 2),
                'Vis 0 [cP]': f'{v0: 5.2e}',
            }
        )

df = pd.DataFrame(data)
df

,compound,x mol,rho [g/ml],E vis [kJ/mol],E dif [kJ/mol],Vis 0 [cP]
0,butanol,100.0,0.83,7.48,8.57,2.70e-03
1,dmeg,0.0,1.07,8.08,9.16,2.29e-03
2,dmeg,4.8,1.07,7.59,8.68,3.19e-03
3,dmeg,9.7,1.05,7.16,8.24,4.14e-03
4,dmeg,16.8,1.03,7.53,8.61,3.12e-03
5,dmeg,19.8,1.02,7.07,8.16,4.27e-03
6,ocm,0.0,1.72,24.33,25.41,1.99e-08
7,ocm,2.4,1.58,21.91,22.99,1.36e-07
8,ocm,5.0,1.57,21.12,22.21,2.20e-07
9,ocm,7.6,1.55,19.45,20.54,7.89e-07


In [ ]:
# Density
x_i=W.w_vol
grouped = DF.groupby(by=["compound"])
fig, ax = plt.subplots()
for (compound,*_), group in grouped:
    name = group["compound"].unique()[0]
    group = group[["rho", x_i.name]].groupby(by=[x_i.name]).mean().reset_index()

    x = group[x_i.name]
    y = group["rho"]

    line_x= np.array(x.to_list()+[100.1])
    line_y = np.array(y.to_list()+[0.83])
    r= linregress(line_x,line_y)
    ax.scatter(x, y, label=f"{compound}")
    ax.plot(line_x,line_x*r.slope+r.intercept)
ax.set_title(f"Rho - {x_i.value}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel("Density [g/ml]")
plt.legend()
plt.show()

In [ ]:
## Grouped compounds Y(x)
# x_i = W.T_1
# y_i = W.ln_Nu
# g_i = W.w_mass

# compound_filter = (DF["x"] == 100) | (DF["x"] == 0)
# data=[]

# fig, ax = plt.subplots(num="DT")
# grouped = DF[compound_filter].groupby(by=['compound'])
# # colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))
# for (compound, *_), group in grouped:
#     y = group[y_i.name]
#     T = group[x_i.name]
#     # color = colors_list.pop()

#     ax.scatter(
#         T,
#         y,
#         s=5,
#         # c=color,
#     )

#     R = linregress(T, y)
#     line_x = np.array([0.00315, 0.0035])
#     ax.plot(
#         line_x,
#         R.slope * line_x + R.intercept,
#         # c=color,
#         linewidth=1,
#         label=f"{compound:5} { R.slope: 5.2e}",
#     )
#     data.append(
#             {
#                 'compound': compound,
#                 'x mol': round(x_val,1),
#                 'E vis [kJ/mol]': round(E_vis, 2),
#                 'E dif [kJ/mol]': round(E_dif, 2),
#                 'Vis 0 [cP]': f'{v0: 5.2e}',
#             }
#         )
# # ax.set_title(compound.upper())
# ax.set_xlabel(x_i.value)
# ax.set_ylabel(y_i.value)

# plt.legend()
# plt.show()